## Iniciando GraspIt!
Para iniciar GraspIt, abre una terminal (Ctrl + Alt + T) y ejecuta el siguiente comando:

``` shellscript
user@pc:~$ roslaunch graspit_interface graspit_interface.launch
```

In [1]:
from practica1.configuration import NOMBRE_GRIPPER_GAZEBO,NOMBRE_GRIPPER,NOMBRE_OBJETO,NOMBRE_OBSTACULO_GAZEBO

In [2]:
# Importar las librarias
from graspit_commander import GraspitCommander
import rospkg
import rospy
from geometry_msgs.msg import Pose


# Obtener la ruta del paquete 'manipulacion_pkg'
# Function: get_path
# Parameters: package name (string)
# Return: path (string)

rospack = rospkg.RosPack()
package_path = rospack.get_path('manipulacion_pkg')

# Inicializar el nodo
if not rospy.get_node_uri():
    rospy.init_node('graspit_commander', anonymous=True, log_level=rospy.WARN)

# Limpiar el mundo actual de GraspIt para un nuevo comienzo
# Function: clearWorld
# Parameters: None
# Return: None
GraspitCommander.clearWorld()

In [3]:
# Define el nombre del gripper a importar

gripper_path = package_path + '/graspit_grippers/' + NOMBRE_GRIPPER + '/'+ \
                NOMBRE_GRIPPER+'.xml'

print(gripper_path)
# Define la pose inicial del gripper
poseHand = Pose()
poseHand.position.z = 0.2
poseHand.position.x = 0.1
poseHand.position.y = 0.0
poseHand.orientation.x = 0.0
poseHand.orientation.y = 0.0
poseHand.orientation.z = 0.0
poseHand.orientation.w = 1

# Importa el gripper a graspit!
# Function: importRobot
# Parameters: gripper_name (string), poseHand (Pose object)
# Return: None
GraspitCommander.importRobot(gripper_path, poseHand)

/workspaces/Handling_RoboticMaster/manipulacion_ws/src/manipulacion_pkg/graspit_grippers/robotiq/robotiq.xml


In [4]:
# Construir la ruta al archivo del objeto
# La ruta incluye la ruta del paquete, el directorio del modelo del objeto 
# y el archivo XML del objeto
graspable_object_path = package_path + '/objects_models/' + \
                        NOMBRE_OBJETO +'/meshes/'+ \
                        NOMBRE_OBJETO +'.xml'


# Inicializar la pose para el objeto agarrable (¡¡NO MODIFICAR!!)
graspable_object_pose = Pose()
graspable_object_pose.position.x = 0.0
graspable_object_pose.position.y = 0.0
graspable_object_pose.position.z = 0.0
graspable_object_pose.orientation.x = 0.0
graspable_object_pose.orientation.y = 0.0
graspable_object_pose.orientation.z = 0.0
graspable_object_pose.orientation.w = 1.0


# Importar el objeto especificado en GraspIt
# Function: importGraspableBody
# Parameters: graspable_object_path (string), pose (Pose object)
# Return: None
GraspitCommander.importGraspableBody(graspable_object_path, 
                                     pose=graspable_object_pose)

In [5]:
from manipulacion_lib import extraer_posicion_superficie_plana_de_yaml
obstacle_pose = Pose()
# Utilizar los valores proporcionados sin modificaciones
position, orientation = extraer_posicion_superficie_plana_de_yaml(package_path + \
                        '/objects_models/obstacle_surface_position.yaml'
                                                    , NOMBRE_OBJETO)
obstacle_pose.position.x = position[0]
obstacle_pose.position.y = position[1]
obstacle_pose.position.z = position[2]
obstacle_pose.orientation.x = orientation[0]
obstacle_pose.orientation.y = orientation[1]
obstacle_pose.orientation.z = orientation[2]
obstacle_pose.orientation.w = orientation[3]
GraspitCommander.importObstacle(package_path+'/objects_models/floor/floor.xml', 
                                pose=obstacle_pose)

In [6]:
from graspit_interface.msg import Planner


# Inicializar el planificador usando el método Simulated Annealing

planner = Planner(Planner.SIM_ANN) 

# Definir el tipo de energía que se utilizará en la búsqueda de agarres
search_energy = "CONTACT_ENERGY" #

# Planificar agarres para el objeto y el gripper importados
# Function: planGrasps
# Parameters: max_steps (int), search_energy (string), planner (Planner object)
# Return: results (GraspPlanningResults object)
results = GraspitCommander.planGrasps(max_steps=32000,
                                      search_energy="CONTACT_ENERGY",
                                      planner=planner)

# Obtener la lista de agarres planificados de los resultados
grasps = results.grasps  # Esta lista contiene los agarres potenciales
                         # generados por GraspIt
# Nota: GraspIt típicamente devuelve un máximo de 20 poses de agarre

In [7]:
# Imprimir el primer agarre y verificar los datos que contiene.
# Asegúrate de que hay al menos un agarre en la lista antes de intentar 
# acceder a él
if grasps:
    first_grasp = grasps[0]  # Accediendo al primer agarre de la lista

    # Imprimir los detalles del primer agarre
    print("Graspable Body ID:", first_grasp.graspable_body_id)
    print("Pose of Gripper:")
    print("  Position:", "x =", first_grasp.pose.position.x, "y =", 
          first_grasp.pose.position.y, "z =", first_grasp.pose.position.z)
    print("  Orientation:", "x =", first_grasp.pose.orientation.x, 
          "y =", first_grasp.pose.orientation.y,
          "z =", first_grasp.pose.orientation.z,
          "w =", first_grasp.pose.orientation.w)
    print("DOFs (Joint Angles of the Gripper):", first_grasp.dofs)
    print("Epsilon Quality (Stability):", first_grasp.epsilon_quality)
    print("Volume Quality (Enclosed Volume):", first_grasp.volume_quality)

else:
    print("No se encontraron poses de agarre.")

Graspable Body ID: 0
Pose of Gripper:
  Position: x = 0.08605980598072924 y = 0.028938920569999596 z = 0.12599277252980323
  Orientation: x = -0.01932105564533515 y = -0.314022174707971 z = 0.05059032723520063 w = 0.9478699221890426
DOFs (Joint Angles of the Gripper): (0.8432625515343983, 0.0, -0.478262551534398, 0.00023585042990040663, 0.6570125515343982, 0.0, -0.425762551534398, -0.00023585042990040663, 0.499512551534398, 0.4487500000000002, -0.7682625515343983)
Epsilon Quality (Stability): -1.0
Volume Quality (Enclosed Volume): 0.0


In [8]:
from graspit_interface.msg import Planner
from graspit_commander import GraspitCommander

# Inicializar el planificador con el método Simulated Annealing (SIM_ANN)
planner = Planner(Planner.SIM_ANN)
# Inicializar una lista vacía para almacenar los mejores agarres
best_grasps = []
# Definir el número mínimo de agarres estables deseado
min_number_of_grasps = 10
# Utilizar un bucle while para seguir planificando agarres hasta alcanzar 
# el mínimo deseado
while len(best_grasps)<min_number_of_grasps:
    # Planificar agarres con GraspitCommander, especificando el número 
    # máximo de pasos y el tipo de energía de búsqueda
    results = GraspitCommander.planGrasps(max_steps=31000,
                                          search_energy="CONTACT_ENERGY",
                                          planner=planner)
    # Iterar a través de los agarres resultantes
    for grasp in results.grasps:
        # Verificar si la calidad épsilon del agarre es mayor a 0.0 
        # (indicativo de un agarre estable)
        if grasp.epsilon_quality > 0.00:
            # Añadir el agarre a la lista de mejores agarres si cumple 
            # con el criterio de estabilidad
            best_grasps.append(grasp)
            # Imprimir el número actual de mejores agarres recolectados
            # para seguimiento
            print("Numero de mejores agarres: ", len(best_grasps))

Numero de mejores agarres:  1
Numero de mejores agarres:  2
Numero de mejores agarres:  3
Numero de mejores agarres:  4
Numero de mejores agarres:  5
Numero de mejores agarres:  6
Numero de mejores agarres:  7
Numero de mejores agarres:  8
Numero de mejores agarres:  9
Numero de mejores agarres:  10


In [9]:
from manipulacion_lib import guardar_grasps_en_yaml
import os

nombre_archivo = 'grasp_poses/''grasp_poses_' + \
                  NOMBRE_GRIPPER + '_' + \
                  NOMBRE_OBJETO+'.yaml'
directorio = os.path.dirname(nombre_archivo)
if directorio and not os.path.exists(directorio):
    os.makedirs(directorio)
    print(f"Carpeta {directorio} creada")
    
guardar_grasps_en_yaml(agarres = best_grasps, nombre_archivo = nombre_archivo)

Carpeta grasp_poses creada
